In [3]:
import pandas as pd
import numpy as np
import geopandas as gpd
import shapefile
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize
from matplotlib.colors import Normalize, ListedColormap
from mpl_toolkits.axes_grid1 import make_axes_locatable

In [17]:
df_school_yr_1 = pd.read_csv("PEN_July_1_2021_June_30_2022.csv", sep=",")
df_school_yr_2 = pd.read_csv("PEN_July_1_2022_December_31_2022.csv", sep=",")
books = df_school_yr_1.append(df_school_yr_2)

florida_books = books.loc[books["State"] == "Florida"]
florida_district_count = florida_books.groupby(by="District")["Title"].count().reset_index()

florida_district_count

# Manually combine districts that repeat
florida_district_count.to_csv("tmp_florda_map_file.csv")

<ipython-input-17-c9d67cd5d4e6>:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  books = df_school_yr_1.append(df_school_yr_2)


In [23]:
final_books = pd.read_csv("florida_school_dist_map/cleaned_florida_schools.csv")
final_books["District"] = final_books["District"].str.lower()
final_books

,Unnamed: 0,District,Title
0,0,brevard,53
1,2,broward,16
2,3,clay,65
3,5,duval,176
4,6,escambia,145
5,7,flagler,17
6,18,hernando,18
7,11,highlands,17
8,12,indian river,161
9,13,jackson,14


In [26]:
shapefile_path = "florida_school_dist_map/tl_2016_12_cousub.shp"
gdf = gpd.read_file(shapefile_path)
gdf["NAME"] = gdf["NAME"].str.lower()
gdf

,STATEFP,COUNTYFP,COUSUBFP,COUSUBNS,GEOID,NAME,NAMELSAD,LSAD,CLASSFP,MTFCC,CNECTAFP,NECTAFP,NCTADVFP,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,geometry
0,12,083,90897,01935751,1208390897,east marion,East Marion CCD,22,Z5,G4040,None,None,None,S,1290236843,102867573,+29.2188682,-081.7943690,"POLYGON ((-81.99520 29.20188, -81.99480 29.202..."
1,12,081,92290,01935861,1208192290,myakka city,Myakka City CCD,22,Z5,G4040,None,None,None,S,810458153,12034777,+27.3686232,-082.2393016,"POLYGON ((-82.50537 27.52341, -82.50340 27.524..."
2,12,081,92600,01935887,1208192600,palmetto,Palmetto CCD,22,Z5,G4040,None,None,None,S,73501608,95271681,+27.5690220,-082.6026768,"POLYGON ((-82.69060 27.59175, -82.67669 27.597..."
3,12,031,91644,02583398,1203191644,jacksonville north,Jacksonville North CCD,22,Z5,G4040,None,None,None,S,528595999,148582518,+30.4840459,-081.5620172,"POLYGON ((-81.83121 30.46745, -81.82340 30.473..."
4,12,031,90110,02583392,1203190110,baldwin,Baldwin CCD,22,Z5,G4040,None,None,None,S,282365593,1088060,+30.2737719,-081.9400913,"POLYGON ((-82.04947 30.18987, -82.04938 30.192..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
311,12,009,92119,01935848,1200992119,melbourne shores-floridana beach,Melbourne Shores-Floridana Beach CCD,22,Z5,G4040,None,None,None,S,17301974,164028578,+27.9301614,-080.4883947,"POLYGON ((-80.56910 28.04185, -80.56590 28.041..."
312,12,009,92132,01935849,1200992132,merritt island,Merritt Island CCD,22,Z5,G4040,None,None,None,S,89968195,107917566,+28.3683349,-080.6858006,"POLYGON ((-80.75790 28.51403, -80.75781 28.515..."
313,12,009,93224,01935937,1200993224,space center,Space Center CCD,22,Z5,G4040,None,None,None,S,373748849,577306315,+28.6162855,-080.6455283,"POLYGON ((-80.84292 28.79115, -80.84187 28.791..."
314,12,089,90364,01935709,1208990364,callahan-hilliard,Callahan-Hilliard CCD,22,Z5,G4040,None,None,None,S,1150932253,2644915,+30.5945177,-081.8889213,"POLYGON ((-82.05033 30.67574, -82.05025 30.676..."


In [34]:
gdf["District"] = gdf["NAME"].str.lower()
map_df = gdf.merge(final_books, on="District", how="left")
map_df["Title"] = map_df["Title"].fillna(0)

map_df.loc[map_df["Title"] != 0]

,STATEFP,COUNTYFP,COUSUBFP,COUSUBNS,GEOID,NAME,NAMELSAD,LSAD,CLASSFP,MTFCC,...,NCTADVFP,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,geometry,District,Unnamed: 0,Title
191,12,115,93107,01935926,1211593107,sarasota,Sarasota CCD,22,Z5,G4040,...,None,S,385892365,133085521,+27.3156119,-082.4497743,"POLYGON ((-82.62489 27.28156, -82.62488 27.283...",sarasota,22.0,3.0
